# Portfolios Example

In [1]:
import load_factors as lf
from factor_portfolio import Factor_Portfolio

In [2]:
assets_list=lf.load_assets([], '../data/')
prices=lf.create_factor_df(assets_list,'close',isForwardFill=False)
returns=lf._convert_prices(prices)

### Momentum algorithm

Create algorithm

In [3]:
lf.create_momentum_factor(assets_list, 21, 252, 'momentum')

In [4]:
algorithm = Factor_Portfolio(assets_list, 'momentum', 15)

In [5]:
algorithm.add_filter('../data/filter/IBX.csv')

Run

In [6]:
algorithm.rebalance_window=1
algorithm.rebalance_period='monthly'
result = algorithm.run(returns)
portfolios = algorithm.df_portfolios

In [8]:
portfolios.to_csv('../portfolios.csv')